<a href="https://colab.research.google.com/github/greatsong/2021dataset/blob/master/2021_%EC%8B%A0%EA%B2%80_%ED%82%A4%EB%AA%B8%EB%AC%B4%EA%B2%8C%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 데이터 수집 - 국방부 공공 데이터
- 국방부에서 제공하는 신체검사 데이터를 수집합니다.  
http://opendata.mnd.go.kr/openinf/sheetview2.jsp?infId=OA-9425
- 판다스로 데이터를 불러오겠습니다.
- 인코딩 문제가 있어서 인코딩 파라미터를 cp949로 주었습니다.

In [ ]:
!git clone https://github.com/greatsong/2021dataset

In [ ]:
cd 2021dataset

In [ ]:
import pandas as pd

df = pd.read_csv('신검데이터.csv', encoding='cp949')
df

- cm, kg 등의 단위가 붙어있는 경우가 있어서 삭제한 버전을 다시 만들었습니다.
- 수정된 데이터를 다시 불러옵니다.

In [ ]:
import pandas as pd

df = pd.read_csv('신검데이터m.csv')
df

### 2. 데이터 살펴보기  
- 데이터를 살펴보겠습니다.
- 키를 x, 몸무게를 y에 저장합니다.

In [ ]:
x = df['신장 센티미터']
y = df['몸무게 킬로그램']

- 스캐터 함수로 상관관계를 살펴보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x, y)
plt.show()

- y축이 몸무게인데 0인 값들이 보이는 것 같습니다.
- 확인해보겠습니다.

In [ ]:
df[df['몸무게 킬로그램'] < 10]

- 진짜 0인 값들이 꽤 있습니다.2016년 9월 23일에 무슨 오류가 있었나봅니다.
- 전체 데이터가 17만건 가까이 되니까 몸무게가 0인 데이터를 삭제하겠습니다.

In [ ]:
idx = df[df['몸무게 킬로그램'] == 0.0].index
idx

- 167983에서 167957로 일부 데이터가 삭제된 것을 볼 수 있습니다.

In [ ]:
df = df.drop(idx)

In [ ]:
df.shape

- 다른 데이터를 살펴보면 가슴 둘레, 머리 둘레에 일부 비어있는 값이 있지만 신장과 몸무게는 이상이 없으므로 넘어갑니다.

In [ ]:
df.info()

- x와 y에 키와 몸무게 값을 다시 저장합니다.
- (TMI : 굳이 이렇게하는 이유는 판다스의 시리즈에 인덱스가 있어서 오류가 나기 때문입니다.)

In [ ]:
x = df['신장 센티미터'].values
y = df['몸무게 킬로그램'].values

In [ ]:
x.shape

In [ ]:
y.shape

- x와 y에 값만 잘 저장된 것을 확인할 수 있습니다.

In [ ]:
x.shape

In [ ]:
x = x.reshape((-1, 1))

In [ ]:
y

- 이상값이 잘 지워졌는지 스캐터 함수를 다시 확인해보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x, y)
plt.show()

- 좋습니다. 그런데 다시 보니 생각보다 120킬로그램이 넘는 사람이 참 많군요?ㅎㅎ
- 히스토그램도 확인해보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
plt.hist(x, bins = 100)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(y, bins = 100)
plt.show()

- 키는 정확히 정규분포를 이루고 있고, 몸무게는 조금 비대칭인 것을 볼 수 있네요.
- 이어서 박스플랏을 그려서 다시 분포를 살펴보겠습니다.

In [ ]:
plt.boxplot(x)
plt.show()

In [ ]:
plt.boxplot(y)
plt.show()

- 키는 175정도가 중위값이고, 몸무게는 70 정도가 중위값인 것 같아보입니다.
- 자 데이터는 전반적으로 잘 살펴보았으니 이제 본격적으로 w와 b값을 찾아보겠습니다.

In [ ]:
# 몸무게 = (키 - 100) * 0.9
# 몸무게 = 키 * 0.9 - 90
w = 0.9
b = -90

- 일반적으로 키에서 100을 뺀 후, 거기에서 0.9를 곱하면 된다는 이야기 들어보셨나요?
- 그 식을 전개하면 w가 0.9, b가 -90이 나오네요.
- 이 식으로 예측선을 그려보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x, y)
plt.plot([150, 200], [150 * w + b,200 * w + b], 'g')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(x, y)
print(model.coef_[0], model.intercept_)

In [ ]:
y_pred = model.predict(x)

plt.scatter(x, y,  color='black')
plt.plot(x, y_pred, color='blue')
plt.plot(x, x * model.coef_[0] + model.intercept_, color='red')
plt.show()

In [ ]:
h = float(input('신검 센터에 오신 것을 환영합니다! :)\n 키를 입력해주세요 : '))
w = float(input('몸무게를 입력해주세요 : '))

std = h * model.coef_[0] + model.intercept_
print(f'표준 몸무게는 {std}이며 {w-std}kg을 감량하시기 바랍니다.')